Adding Wind Pressure Coefficients to Surfaces in an E+ IDF File using Python
==================================================================================================

Introduction
------------

In this tutorial, we will walk you through a Python script designed to add wind pressure coefficients to surfaces in an EnergyPlus IDF file. This can be particularly useful when you're modelling airflow and want to understand how the building's geometry interacts with the wind.

Prerequisites
-------------

Before we begin, ensure you have the following:

1.  Basic knowledge of Python.
2.  EnergyPlus and the `Energy+.idd` file.
3.  Python's EPPY library installed. If not, you can install it using pip: `pip install eppy`.


The `Energy+.idd` file, often simply referred to as the IDD, is the Input Data Dictionary for EnergyPlus. This file defines all the objects and fields that an IDF (Input Data File) will have. The IDD is crucial for software and scripts (like the one using EPPY in your previous query) to understand the structure of an IDF.

Here's how you can find the `Energy+.idd` file:

1.  Installation Directory: When you install EnergyPlus, the IDD is usually placed in the main installation directory of EnergyPlus.

    For example, on a Windows system, if you've installed EnergyPlus 9.3, the path might be something like:

    makefileCopy code

    `C:\EnergyPlusV9-3-0\EnergyPlusIDD.idd`

    On Linux, it might be under `/usr/local/EnergyPlus-9-3-0/` or a similar location, depending on how you've installed it.

    On macOS, if you've installed EnergyPlus using the installer package, the path might be:

    Copy code

    `/Applications/EnergyPlus-9-3-0/`

2.  EnergyPlus Toolkits: Some toolkits or GUI-based applications that work with EnergyPlus might keep their own copy of the IDD. If you're using such a toolkit, you might find the IDD in its directories.

3.  Search Your System: If you're unsure where EnergyPlus was installed, you can use system search functions:

    -   On Windows, you can use the search bar in the File Explorer.
    -   On Linux, you can use commands like `find` or `locate` (if `updatedb` has been run previously).
    -   On macOS, you can use Spotlight or Finder.
4.  Check Documentation: The EnergyPlus documentation might provide information on the installation directory, which can guide you to the location of the IDD.

When referencing the IDD in scripts or software, it's good practice to always point to the version of the IDD that matches the version of your EnergyPlus IDF files. This ensures compatibility and that all objects and fields are recognized correctly.



The Script
----------

The script you'll be working with uses the EPPY library to interact with EnergyPlus IDF files. We'll start by importing necessary modules, setting paths, and then diving into the core logic of the script.

In [1]:
from update_cp import *

In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Adding CP Values to IDF')

# Constants
IDF_PATH = 'data/in.idf'
IDD_PATH = 'idd/Energy+_22_2_0.idd'
CP_PATH = 'data/Cp.csv'

In [4]:
path_checker(IDF_PATH, IDD_PATH, CP_PATH)
logger.info(f'Setting IDD file to {IDD_PATH}')
IDF.setiddname(IDD_PATH)    # Setting the IDD file so EPPY knows how to read the IDF file.
logger.info(f'Loading IDF file from {IDF_PATH}')
idf1 = IDF(IDF_PATH)
update_idf_params(idf1)
wind_pressure_coefficient_array = add_wind_pressure_coefficients_array(idf1)
kdtree, coords_df = vectorize_points()
building_objects, fenestration_objects = fetch_surfaces(idf1)
add_cp_fenestration(idf1, fenestration_objects, wind_pressure_coefficient_array, kdtree=kdtree, coords_df=coords_df)
building_objects_no_external_node_name = get_surface_no_node(idf1,building_objects)
add_cp_surfaces(idf1, building_objects_no_external_node_name, wind_pressure_coefficient_array, kdtree=kdtree, coords_df=coords_df)
# Set AirflowNetwork:SimulationControl object
logger.info('Setting AirflowNetwork:SimulationControl object to use input wind pressure coefficients')
idf1.idfobjects['AirflowNetwork:SimulationControl'][0].Wind_Pressure_Coefficient_Type = 'Input'
logger.info('Saving IDF file with CP')
idf1.saveas('in_with_cp.idf')
logger.info('Done!')

INFO:Adding CP Values to IDF:Checking paths
INFO:Adding CP Values to IDF:All paths are valid
INFO:Adding CP Values to IDF:Setting IDD file to idd/Energy+_22_2_0.idd
INFO:Adding CP Values to IDF:Loading IDF file from data/in.idf
INFO:Adding CP Values to IDF:Updating IDF to use JtoKWH
INFO:Adding CP Values to IDF:Saving original IDF file
INFO:Adding CP Values to IDF:Adding wind pressure coefficients array
INFO:Adding CP Values to IDF:Vectorizing points from data/Cp.csv
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:Adding CP Values to IDF:Translating points
INFO:Adding CP Values to IDF:Building KDTree
INFO:Adding CP Values to IDF:Fetching surfaces
INFO:Adding CP Values to IDF:Adding CP for fenestration


INFO:Adding CP Values to IDF:Getting surfaces with no node setup
INFO:Adding CP Values to IDF:Adding CP for surfaces


INFO:Adding CP Values to IDF:Setting AirflowNetwork:SimulationControl object to use input wind pressure coefficients
INFO:Adding CP Values to IDF:Saving IDF file with CP


INFO:Adding CP Values to IDF:Done!
